# CNIT 484 Final Project #
## Financial Analysis ##

### Libraries ###
Add more as needed. Make sure to add to requirements.txt as well.

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

## Loading Data ##

In [26]:
raw_data = pd.read_csv("../Datasets/1_ETHUSDT_1.1.2018-1.2.2024_1hour.csv", header=None)
print(raw_data.head())
print(raw_data.shape)


                    0       1       2       3       4           5   \
0  2017-12-31 19:00:00  733.01  734.52  720.03  727.62  2105.90100   
1  2017-12-31 20:00:00  727.01  732.00  716.80  717.97  2305.97086   
2  2017-12-31 21:00:00  717.67  725.75  717.59  724.05  2166.45725   
3  2017-12-31 22:00:00  723.95  737.99  722.70  734.50  2160.90450   
4  2017-12-31 23:00:00  734.99  744.98  730.01  744.82  2335.33705   

              6             7     8           9              10  11  
0  1514768399999  1.528559e+06  3114  1275.23271  925445.068280   0  
1  1514771999999  1.675753e+06  2875  1035.33513  753211.787422   0  
2  1514775599999  1.564151e+06  2957  1179.82843  851942.067625   0  
3  1514779199999  1.582200e+06  3647  1095.63271  801470.072777   0  
4  1514782799999  1.724788e+06  3512  1313.03081  970430.311743   0  
(53207, 12)


In [27]:
raw_data.columns = ['date', 'open', 'high', 'low', 'close', 'volume'] + list(raw_data.columns[6:])
raw_data.drop(raw_data.columns[6:], axis=1, inplace=True)
print(raw_data.head())
print(raw_data.shape)


                  date    open    high     low   close      volume
0  2017-12-31 19:00:00  733.01  734.52  720.03  727.62  2105.90100
1  2017-12-31 20:00:00  727.01  732.00  716.80  717.97  2305.97086
2  2017-12-31 21:00:00  717.67  725.75  717.59  724.05  2166.45725
3  2017-12-31 22:00:00  723.95  737.99  722.70  734.50  2160.90450
4  2017-12-31 23:00:00  734.99  744.98  730.01  744.82  2335.33705
(53207, 6)


## Preprocessing ##

In [32]:
# Separate the date column
date_column = raw_data.iloc[:, 0]

# Normalize all columns except the first one
scaler = MinMaxScaler()
normalized_data = pd.DataFrame(scaler.fit_transform(raw_data.iloc[:, 1:]), columns=raw_data.columns[1:])

# Concatenate the date column back
normalized_data = pd.concat([date_column, normalized_data], axis=1)

print(normalized_data.head())

                  date      open      high       low     close    volume
0  2017-12-31 19:00:00  0.136596  0.136168  0.134327  0.135470  0.004270
1  2017-12-31 20:00:00  0.135337  0.135641  0.133647  0.133444  0.004675
2  2017-12-31 21:00:00  0.133377  0.134335  0.133813  0.134720  0.004392
3  2017-12-31 22:00:00  0.134695  0.136893  0.134889  0.136914  0.004381
4  2017-12-31 23:00:00  0.137012  0.138354  0.136427  0.139080  0.004735
